In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

2024-08-12 08:49:01.196460: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 08:49:01.197170: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 08:49:01.202627: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 08:49:01.216623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 08:49:01.238714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
def preprocess_image(lr_img, hr_img):
    hr_img = tf.image.central_crop(hr_img, central_fraction=0.5)
    hr_img = tf.image.resize(hr_img, (224, 224))
    lr_img = tf.image.resize(hr_img, (112, 112))
    lr_img = tf.image.resize(lr_img, (224, 224))
    return lr_img, hr_img

def load_div2k_dataset(num_samples=100):
    dataset = tfds.load('div2k', split='train', as_supervised=True)
    dataset = dataset.take(num_samples)
    lr_images = []
    hr_images = []
    for lr_img, hr_img in tfds.as_numpy(dataset):
        lr_img, hr_img = preprocess_image(lr_img, hr_img)
        lr_images.append(lr_img)
        hr_images.append(hr_img)
    return np.array(lr_images), np.array(hr_images)

lr_images, hr_images = load_div2k_dataset()

2024-08-12 08:49:13.023100: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
def split_data(lr_images, hr_images, train_ratio=0.6, val_ratio=0.2):
    lr_train, lr_temp, hr_train, hr_temp = train_test_split(lr_images, hr_images, train_size=train_ratio)
    val_size = val_ratio / (1 - train_ratio)
    lr_val, lr_test, hr_val, hr_test = train_test_split(lr_temp, hr_temp, train_size=val_size)
    return lr_train, lr_val, lr_test, hr_train, hr_val, hr_test

lr_train, lr_val, lr_test, hr_train, hr_val, hr_test = split_data(lr_images, hr_images)

In [5]:
def psnr_metric(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=255.0)

In [6]:
def build_srcnn_model():
    input_layer = layers.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, (9, 9), activation='relu', padding='same')(input_layer)
    x = layers.Conv2D(32, (5, 5), activation='relu', padding='same')(x)
    x = layers.Conv2D(3, (5, 5), activation='linear', padding='same')(x)
    output_layer = layers.Add()([input_layer, x])
    model = models.Model(inputs=input_layer, outputs=output_layer)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=10000,
    decay_rate=0.9)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss='mean_squared_error', metrics=[psnr_metric])
    return model

srcnn_model = build_srcnn_model()

In [7]:
def build_srcnn_reg_model(reg_factor=1e-4):
    model = models.Sequential([
        layers.Conv2D(64, (9, 9), activation='relu', padding='same', kernel_regularizer=regularizers.l2(reg_factor), input_shape=(224, 224, 3)),
        layers.Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=regularizers.l2(reg_factor)),
        layers.Conv2D(3, (5, 5), activation='linear', padding='same')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=[psnr_metric])
    return model

srcnn_reg_model = build_srcnn_reg_model()

/home/miljan/Image-Super-Resolution/my_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
def build_srcnn_tanh_model():
    model = models.Sequential([
        layers.Input(shape=(224, 224, 3)),
        layers.Lambda(lambda x: (x / 127.5) - 1.0),  # Normalize to [-1, 1]
        layers.Conv2D(64, (9, 9), activation='relu', padding='same'),
        layers.Conv2D(32, (5, 5), activation='relu', padding='same'),
        layers.Conv2D(3, (5, 5), activation='linear', padding='same'),
        layers.Lambda(lambda x: (x + 1.0) * 127.5)  # De-normalize to [0, 255]
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=[psnr_metric])
    return model

srcnn_tanh_model = build_srcnn_tanh_model()

In [9]:
def build_srcnn_batchnorm_model():
    model = models.Sequential([
        layers.Conv2D(64, (9, 9), padding='same', input_shape=(224, 224, 3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (5, 5), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(3, (5, 5), activation='linear', padding='same')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=[psnr_metric])
    return model

srcnn_batchnorm_model = build_srcnn_batchnorm_model()